# TUTORIAL: Modelado de Niveles Freáticos con Pastas

Este notebook demuestra cómo construir modelos de niveles freáticos usando la biblioteca Pastas, considerando múltiples factores de estrés (precipitación, evaporación, y bombeos).

**Estructura:**
1. Carga y preparación de datos de niveles
2. Análisis de tendencias (Mann-Kendall)
3. Carga y preparación de datos de estrés (clima y bombeos)
4. Construcción de modelos Pastas con diferentes configuraciones
5. Evaluación y exportación de resultados

## 1. Imports y configuración

In [ ]:
import os
import time
import warnings
from multiprocessing import Pool, cpu_count
import pandas as pd
import pastas as ps
import matplotlib.pyplot as plt
import pymannkendall as mk
import numpy as np
from scipy import stats

warnings.filterwarnings("ignore", category=DeprecationWarning)
ps.set_log_level("ERROR")
os.makedirs("plots", exist_ok=True)

# Multiprocessing: 1 = secuencial, >1 = paralelo por pozo (para cada modelo)
N_WORKERS = 1  # ej. 4 o cpu_count() - 1 para acelerar

## 2.1 Funciones auxiliares

In [ ]:
def load_pumping_data(filepath: str) -> pd.Series:
    """Carga datos de bombeo mensuales y los convierte a serie diaria (m³/día, negativos)."""
    df = pd.read_csv(filepath)
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d/%m/%Y", errors="coerce")
    df = df.dropna().set_index("Fecha").sort_index()
    if df.index.duplicated().any():
        df = df.groupby(df.index).mean(numeric_only=True)
    days_in_month = df.index.days_in_month
    df["m3_day"] = df["total"] / days_in_month
    daily_series = df["m3_day"].resample("D").ffill()
    return -1 * daily_series


def distancia_utm(este_1, norte_1, este_2, norte_2):
    """Distancia euclidiana entre dos puntos UTM (metros)."""
    e1, n1 = np.asarray(este_1, dtype=float), np.asarray(norte_1, dtype=float)
    e2, n2 = np.asarray(este_2, dtype=float), np.asarray(norte_2, dtype=float)
    return np.hypot(e2 - e1, n2 - n1)


def calculate_model_statistics(model: ps.Model) -> dict:
    """Estadísticas de validación del modelo (R², RMSE, tests sobre residuales)."""
    residuals = model.residuals()
    t_stat, p_value_t = stats.ttest_1samp(residuals, 0)
    stat_wilc, p_value_wilc = stats.wilcoxon(residuals)
    stat_shap, p_value_shap = stats.shapiro(residuals)
    r2 = model.stats.rsq()
    rmse = model.stats.rmse()
    bic = model.stats.bic()
    obs = model.oseries.series
    sim = model.simulate()
    std_obs = model.oseries.series.dropna().std()
    mae = np.mean(np.abs(sim - obs))
    return {
        "R²": r2, "RMSE": rmse, "std_obs07": std_obs * 0.7,
        "p-value wilconox": p_value_wilc, "MAE": mae, "BIC": bic, "p-value shapiro": p_value_shap,
    }


def calculate_ivm(row: pd.Series) -> str:
    """Indicador de Validación del Modelo: Válido si cumple R², RMSE, Wilcoxon, MAE."""
    try:
        conditions = [
            pd.notna(row["R²"]) and row["R²"] > 0.65,
            pd.notna(row["RMSE"]) and row["RMSE"] < row["std_obs07"],
            pd.notna(row["p-value wilconox"]) and row["p-value wilconox"] > 0.05,
            pd.notna(row["MAE"]) and row["MAE"] < row["std_obs07"],
        ]
        return "Válido" if all(conditions) else "No Válido"
    except (KeyError, TypeError):
        return "No Válido"


def _trunc3(x: float) -> float:
    return float(np.trunc(float(x) * 1000.0) / 1000.0)


def _mk_table(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica Mann-Kendall a cada pozo; retorna tabla con Z, S, p-value, tendencia."""
    def _trend_es(t):
        t = str(t).strip().lower()
        if t == "increasing": return "Creciente"
        if t == "decreasing": return "Decreciente"
        return "Sin tendencia"
    mk_results = []
    for pozo, g in df.groupby("Pozo", sort=True):
        serie = g.sort_values("Fecha")["Valor"].dropna()
        if len(serie) < 3:
            continue
        res = mk.original_test(serie)
        mk_results.append({"Pozo": pozo, "Estadistico Z": _trunc3(res.z), "Estadistico S": int(res.s),
                          "p-value": _trunc3(res.p), "tendencia": _trend_es(res.trend)})
    return pd.DataFrame(mk_results)

## 2.2 Funciones para construir modelos

In [ ]:
def get_data_bundle():
    """Agrupa datos necesarios para construir modelos (se usa en secuencial y paralelo)."""
    return {
        "niveles_series": niveles_series,
        "coordenadas_niveles": coordenadas_niveles,
        "coordenadas_bombeos": coordenadas_bombeos,
        "sm_precip": sm_precip, "sm_evap": sm_evap,
        "ALB_series": ALB_series, "SOP_series": SOP_series, "MOP_series": MOP_series,
        "TIL_series": TIL_series, "TUC_series": TUC_series, "PEINE_series": PEINE_series,
    }


def create_model_with_data(modelo: str, location_names: list, data: dict):
    """Construye y calibra un modelo Pastas para cada pozo usando solo `data`. Returns (model_stats, gains_data)."""
    ns = data["niveles_series"]
    coord_n, coord_b = data["coordenadas_niveles"], data["coordenadas_bombeos"]
    sm_prec, sm_ev = data["sm_precip"], data["sm_evap"]
    ALB = data["ALB_series"]
    SOP, MOP = data["SOP_series"], data["MOP_series"]
    TIL, TUC, PEINE = data["TIL_series"], data["TUC_series"], data["PEINE_series"]
    model_stats = {}
    gains_data = []
    for pozo in location_names:
        datos = ns[ns["Pozo"] == pozo]["Valor"]
        datos.index = pd.to_datetime(datos.index, errors="coerce")
        datos = datos.dropna().sort_index().resample("D").mean()
        ml = ps.Model(datos, name=f"{pozo} - {modelo}")
        ml.add_stressmodel(sm_prec)
        ml.add_stressmodel(sm_ev)
        if modelo == "Modelo_B":
            sm_alb = ps.StressModel(ALB, ps.Hantush(), name="albemarle", up=False, settings="well")
            ml.add_stressmodel(sm_alb)
        elif modelo == "Modelo_C":
            este_pozo = float(coord_n.loc[pozo]["Este"])
            norte_pozo = float(coord_n.loc[pozo]["Norte"])
            bombeos, series = ["alb", "til", "tuc", "peine"], [ALB, TIL, TUC, PEINE]
            dist = [distancia_utm(float(coord_b.loc[b]["Este"]), float(coord_b.loc[b]["Norte"]), este_pozo, norte_pozo) for b in bombeos]
            ml.add_stressmodel(ps.WellModel(series, "WellModel", dist))
        elif modelo == "Modelo_D":
            este_pozo = float(coord_n.loc[pozo]["Este"])
            norte_pozo = float(coord_n.loc[pozo]["Norte"])
            bombeos, series = ["sop", "mop"], [SOP, MOP]
            dist = [distancia_utm(float(coord_b.loc[b]["Este"]), float(coord_b.loc[b]["Norte"]), este_pozo, norte_pozo) for b in bombeos]
            ml.add_stressmodel(ps.WellModel(series, "WellModel", dist))
        elif modelo == "Modelo_E":
            este_pozo = float(coord_n.loc[pozo]["Este"])
            norte_pozo = float(coord_n.loc[pozo]["Norte"])
            bombeos = ["alb", "sop", "mop", "til", "tuc", "peine"]
            series = [ALB, SOP, MOP, TIL, TUC, PEINE]
            dist = [distancia_utm(float(coord_b.loc[b]["Este"]), float(coord_b.loc[b]["Norte"]), este_pozo, norte_pozo) for b in bombeos]
            ml.add_stressmodel(ps.WellModel(series, "WellModel", dist))
        ml.add_noisemodel(ps.NoiseModel())
        ml.solve(report=False)
        model_stats[pozo] = calculate_model_statistics(ml)
        os.makedirs(modelo, exist_ok=True)
        fig, axes = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={"width_ratios": [4, 1]})
        ml.plot(ax=axes[0])
        axes[0].set_ylabel("Nivel de agua [msnm]")
        axes[0].set_title(f"{modelo} - {pozo}")
        axes[0].ticklabel_format(axis="y", style="plain", useOffset=False)
        axes[1].hist(ml.residuals(), bins=25, edgecolor="black", orientation="horizontal")
        axes[1].set_xlabel("Frecuencia")
        axes[1].set_ylabel("Residuales")
        axes[1].set_title("Residuales")
        plt.tight_layout()
        fig.savefig(os.path.join(modelo, f"{modelo}_{pozo}.png"), dpi=200)
        plt.close()
        if modelo in ["Modelo_C", "Modelo_D", "Modelo_E"]:
            wm = ml.stressmodels["WellModel"]
            names = {"Modelo_C": ["alb", "til", "tuc", "peine"], "Modelo_D": ["sop", "mop"]}.get(modelo, ["alb", "sop", "mop", "til", "tuc", "peine"])
            pozo_gains = {"pozo": pozo}
            for i, name in enumerate(names[: len(wm.stress)]):
                p = wm.get_parameters(model=ml, istress=i)
                pozo_gains[name] = wm.rfunc.gain(p) * 1e6 / 365.25
            gains_data.append(pozo_gains)
    return model_stats, gains_data


def _run_one_pozo_for_model(modelo: str, pozo: str, bundle: dict):
    """Wrapper para multiprocessing: un modelo para un pozo. Returns (modelo, model_stats, gains_data)."""
    model_stats, gains_data = create_model_with_data(modelo, [pozo], bundle)
    return (modelo, model_stats, gains_data)

## 3. Carga y preparación de datos de niveles

In [ ]:
t_start = time.perf_counter()
print("=== 3. CARGANDO DATOS DE NIVELES ===")

niveles_path = os.path.join("datos", "pozos nivel peine", "Niveles.csv")
niveles = pd.read_csv(niveles_path, encoding="utf-8-sig")
niveles["Fecha"] = pd.to_datetime(niveles["Fecha"], dayfirst=True, errors="coerce")
niveles["Valor"] = pd.to_numeric(niveles["Valor"], errors="coerce")
niveles["Pozo"] = niveles["Pozo"].astype(str).str.strip()
niveles["Tipo"] = niveles["Tipo"].astype(str).str.strip().str.upper()
niveles = niveles.dropna(subset=["Fecha", "Pozo", "Tipo", "Valor"]).copy()
inicio = pd.Timestamp("2000-01-01")
niveles = niveles[niveles["Fecha"] >= inicio].copy()

monitoreo_path = os.path.join("datos", "pozos nivel peine", "monitoreo_total.csv")
monitoreo = pd.read_csv(monitoreo_path, encoding="utf-8-sig")
allowed = monitoreo["Nombre"].astype(str).str.strip().str.replace(r"^Pozo\s+", "", regex=True).dropna().unique()
niveles = niveles[niveles["Pozo"].isin(set(allowed))].copy()

print(f"Total de registros de niveles: {len(niveles)}")
print(f"Pozos únicos: {niveles['Pozo'].nunique()}")

## 4. Análisis de tendencias (Mann-Kendall)

In [ ]:
print("=== 4. ANÁLISIS DE TENDENCIAS ===")

exclude_head = {"PP-03", "MP-07C-1", "MP-07A", "L10-1", "PP-01", "MP-08A", "BA-30"}
mk_head = _mk_table(niveles[(niveles["Tipo"] == "HEAD") & (~niveles["Pozo"].isin(exclude_head))])
mk_head_excl = _mk_table(niveles[(niveles["Tipo"] == "HEAD") & (niveles["Pozo"].isin(exclude_head))])
mk_stage = _mk_table(niveles[niveles["Tipo"] == "STAGE"])

mk_head.to_csv("resultados_mann_kendall_head.csv", index=False, float_format="%.3f")
mk_head_excl.to_csv("resultados_mann_kendall_head_excluidos.csv", index=False, float_format="%.3f")
mk_stage.to_csv("resultados_mann_kendall_limnimetricos.csv", index=False, float_format="%.3f")

print(f"MK HEAD: {len(mk_head)} pozos")
print(f"MK HEAD excluidos: {len(mk_head_excl)} pozos")
print(f"MK STAGE: {len(mk_stage)} pozos")

## 5. Carga de datos de estrés (clima y bombeos)

In [ ]:
print("=== 5. CARGANDO DATOS DE ESTRÉS ===")

archivos_precipitacion = [
    os.path.join('datos', 'resultados_meteo', 'precip', 'Prec_CHAXA.csv'),
    os.path.join('datos', 'resultados_meteo', 'precip', 'Prec_LZA9-1 (Interna).csv'),
    os.path.join('datos', 'resultados_meteo', 'precip', 'Prec_LZA10-1.csv')
]
archivos_evaporacion = [
    os.path.join('datos', 'resultados_meteo', 'evap', 'Evap_CHAXA.csv'),
    os.path.join('datos', 'resultados_meteo', 'evap', 'Evap_LZA9-1 (Interna).csv')
]
precipitacion_list = [pd.read_csv(a, index_col=0, parse_dates=True).squeeze("columns") for a in archivos_precipitacion]
precipitacion = pd.concat(precipitacion_list).groupby(level=0).mean()
evaporacion_list = [pd.read_csv(a, index_col=0, parse_dates=True).squeeze("columns") for a in archivos_evaporacion]
evaporacion = pd.concat(evaporacion_list).groupby(level=0).mean()

precipitacion.index = pd.to_datetime(precipitacion.index, errors="coerce")
precipitacion = precipitacion.dropna().sort_index()
evaporacion.index = pd.to_datetime(evaporacion.index, errors="coerce")
evaporacion = evaporacion.dropna().sort_index()
prec = precipitacion.resample("D").sum()
evap = evaporacion.resample("D").mean()

coef = 0.225
sm_precip = ps.StressModel(prec * coef, ps.Gamma(), settings="prec", name="precipitacion")
sm_evap = ps.StressModel(evap, ps.Gamma(), settings="evap", name="evaporacion")

ALB_series = load_pumping_data(os.path.join("datos", "pumping", "alb_pump.csv"))
SOP_series = load_pumping_data(os.path.join("datos", "pumping", "SOP_monthly_m3.csv"))
MOP_series = load_pumping_data(os.path.join("datos", "pumping", "MOP_monthly_m3.csv"))
TIL_series = load_pumping_data(os.path.join("datos", "pumping", "tilopozo_pumping.csv"))
TUC_series = load_pumping_data(os.path.join("datos", "pumping", "tucucaro_pumping.csv"))
PEINE_series = load_pumping_data(os.path.join("datos", "pumping", "Pozo_peine_pumping.csv"))

print("Datos de estrés cargados correctamente")

## 6. Carga de coordenadas

In [ ]:
print("=== 6. CARGANDO COORDENADAS ===")

coordenadas_bombeos_path = os.path.join("datos", "pumping", "bombeos_ubicacion.csv")
coordenadas_bombeos = pd.read_csv(coordenadas_bombeos_path, encoding="utf-8-sig")
coordenadas_bombeos.set_index("Nombre", inplace=True)

coordenadas_niveles_path = os.path.join("datos", "pozos nivel peine", "monitoreo_total.csv")
coordenadas_niveles = pd.read_csv(coordenadas_niveles_path, encoding="utf-8-sig")
if "Nombre" in coordenadas_niveles.columns:
    coordenadas_niveles.set_index("Nombre", inplace=True)
else:
    col_name = coordenadas_niveles.columns[coordenadas_niveles.columns.str.lower() == "nombre"][0]
    coordenadas_niveles.set_index(col_name, inplace=True)

print(f"Coordenadas de {len(coordenadas_bombeos)} bombeos cargadas")
print(f"Coordenadas de {len(coordenadas_niveles)} pozos cargadas")

## 7. Ejecución: modelos secuencial (A, B, C...), pozos en paralelo

In [ ]:
Modelos = ["Modelo_A", "Modelo_B", "Modelo_C", "Modelo_D", "Modelo_E"]
niveles_series = niveles.copy()
niveles_series.set_index("Fecha", inplace=True)
location_names = sorted(niveles["Pozo"].unique().tolist())
bundle = get_data_bundle()

total_modelos = 0
for modelo in Modelos:
    print(f"\n--- {modelo} ---")
    if N_WORKERS <= 1:
        results = []
        for pozo in location_names:
            results.append(_run_one_pozo_for_model(modelo, pozo, bundle))
    else:
        with Pool(N_WORKERS) as pool:
            results = pool.starmap(_run_one_pozo_for_model, [(modelo, pozo, bundle) for pozo in location_names])
    model_stats = {}
    gains_data = []
    for (m, stats, gains) in results:
        model_stats.update(stats)
        gains_data.extend(gains)
    total_modelos += len(model_stats)
    df_stats = pd.DataFrame.from_dict(model_stats, orient="index")
    df_stats["IVM"] = df_stats.apply(calculate_ivm, axis=1)
    summary_row = pd.DataFrame([{c: "" for c in df_stats.columns}], index=["Summary"])
    summary_row["IVM"] = f"Total Válidos: {(df_stats['IVM'] == 'Válido').sum()}"
    df_stats = pd.concat([df_stats, summary_row])
    df_stats.to_csv(f"Summary_{modelo}.csv")
    print(f"  Guardado: Summary_{modelo}.csv")
    if modelo in ["Modelo_C", "Modelo_D", "Modelo_E"] and gains_data:
        df_gains = pd.DataFrame(gains_data).set_index("pozo")
        df_gains = pd.concat([df_gains, pd.DataFrame([df_gains.mean()], index=["mean"])])
        df_gains.to_csv(f"{modelo}_gains_by_pozo.csv")
        print(f"  Guardado: {modelo}_gains_by_pozo.csv")

elapsed = time.perf_counter() - t_start
print("\n=== PROCESO COMPLETADO ===")
print(f"Total de modelos creados: {total_modelos}")
if elapsed >= 60:
    print(f"Tiempo total: {int(elapsed // 60)}m {elapsed % 60:.1f}s")
else:
    print(f"Tiempo total: {elapsed:.1f}s")